In [116]:
# Setup Boto3 client, get IAM role ARN
rolename  = DWH_ROLE_NAME
role_desc = "Allows Redshift clusters to call AWS services on your behalf."
principal = {'Service': 'redshift.amazonaws.com'}
version   = '2012-10-17'

try:
    iam = boto3.client('iam',
                           region_name="us-west-2",
                           aws_access_key_id=KEY,
                           aws_secret_access_key=SECRET
                           )
    log.info('Boto3 IAM client created')
    created_role = iam.create_role(
        Path='/',
        RoleName=rolename,
        Description = role_desc,
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': principal }],
             'Version': version})
    )
    log.info('Role "%s" created successfully', created_role)
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        log.warning('Role already exists')
    else:
        log.error('Failed to create Boto3 client, reason: %s', e)
try:
    iam.attach_role_policy(RoleName=DWH_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']
    roleArn = iam.get_role(RoleName=DWH_ROLE_NAME)['Role']['Arn']
    log.info('Attached policy and received role ARN "%s"', roleArn)
except:
    log.error('Attaching policy failed, no ARN received')

2021-02-20 19:52:32 INFO 	 Boto3 IAM client created 
2021-02-20 19:52:32 WARNING 	 Role already exists 
2021-02-20 19:52:33 INFO 	 Attached policy and received role ARN "arn:aws:iam::422675603730:role/dwhRole" 


In [117]:
# Start Redshift Instance

# Create Boto Client
try:
    redshift = boto3.client('redshift',
                           region_name="us-west-2",
                           aws_access_key_id=KEY,
                           aws_secret_access_key=SECRET
                           )
    log.info('Successfully prepared Redshift Client')
except Exception as e:
    log.error('Failed to prepare Redshift Client with error %s', e)

# Create Cluster
try:
    log.info('Creating cluster %s', DWH_CLUSTER_IDENTIFIER)
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        #NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]
    )
    log.info('Cluster startup in progress %s', response)
except Exception as e:
    if e.response['Error']['Code'] == 'ClusterAlreadyExists':
        log.warning('Cluster already exists')
    else:
        log.error('Failed to start Cluster with error %s', e)

# Prettyfier for Redshift cluster properties
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', None)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId',]
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])


# Wait for Redshift instance to come up
redshift_waiter = redshift.get_waiter('cluster_available')
redshift_waiter.wait(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)

# Print properties, log endpoint DNS name
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
log.info(myClusterProps)
log.info('Cluster Endpoint DNS name "%s"', myClusterProps['Endpoint']['Address'])

# Open TCP Connection
try:
    ec2 = boto3.resource('ec2',
                           region_name="us-west-2",
                           aws_access_key_id=KEY,
                           aws_secret_access_key=SECRET
                           )
    log.info('Successfully prepared EC2 Client')
except Exception as e:
    log.error('Failed to prepare EC2 with error %s', e)

# Implement ALLOW for incoming TCP connections on specified port
# Get our cluster's security group
sec_group = myClusterProps['VpcSecurityGroups'][0]['VpcSecurityGroupId']
logging.info('Cluster VPC security group is %s', sec_group)

# Authorize TCP rule for this security group
try:
    log.info('Implementing TCP Rule')
    security_group = ec2.SecurityGroup(sec_group)
    response = security_group.authorize_ingress(
        GroupName=security_group.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
    log.info('TCP rule successfully implemented for %s', response)
except Exception as e:
    if e.response['Error']['Code'] == 'InvalidPermission.Duplicate':
        log.warning('TCP rule already exists')
    else:
        log.error('Failed to prepare Redshift Client with error %s', e)
        

2021-02-20 19:52:34 INFO 	 Successfully prepared Redshift Client 
2021-02-20 19:52:34 INFO 	 Creating cluster dwhClusterSTS 
2021-02-20 19:52:36 INFO 	 Cluster startup in progress {'Cluster': {'ClusterIdentifier': 'dwhclustersts', 'NodeType': 'dc2.large', 'ClusterStatus': 'creating', 'ClusterAvailabilityStatus': 'Modifying', 'MasterUsername': 'dwhuser', 'DBName': 'dwh', 'AutomatedSnapshotRetentionPeriod': 1, 'ManualSnapshotRetentionPeriod': -1, 'ClusterSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0ea5b83e1b504afb2', 'Status': 'active'}], 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0', 'ParameterApplyStatus': 'in-sync'}], 'ClusterSubnetGroupName': 'default', 'VpcId': 'vpc-0f99910ec19fe058b', 'PreferredMaintenanceWindow': 'fri:10:00-fri:10:30', 'PendingModifiedValues': {'MasterUserPassword': '****'}, 'ClusterVersion': '1.0', 'AllowVersionUpgrade': True, 'NumberOfNodes': 1, 'PubliclyAccessible': True, 'Encrypted': False, 'Tags': [], 'Enha

In [115]:
# Clean up and remove cluster w/o snapshot
response = redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
log.info('Cluster is being deleted with %s', response)
#response = iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
log.info('IAM Policy is being detached with %s', response)
#iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
log.info('Role is being deleted with %s', response)


2021-02-20 12:56:31 INFO 	 Cluster is being deleted with {'Cluster': {'ClusterIdentifier': 'dwhclustersts', 'NodeType': 'dc2.large', 'ClusterStatus': 'deleting', 'ClusterAvailabilityStatus': 'Modifying', 'MasterUsername': 'dwhuser', 'DBName': 'dwh', 'Endpoint': {'Address': 'dwhclustersts.cdzpwfcnkher.us-west-2.redshift.amazonaws.com', 'Port': 5439}, 'ClusterCreateTime': datetime.datetime(2021, 2, 20, 10, 2, 11, 37000, tzinfo=tzutc()), 'AutomatedSnapshotRetentionPeriod': 1, 'ManualSnapshotRetentionPeriod': -1, 'ClusterSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0ea5b83e1b504afb2', 'Status': 'active'}], 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0', 'ParameterApplyStatus': 'in-sync'}], 'ClusterSubnetGroupName': 'default', 'VpcId': 'vpc-0f99910ec19fe058b', 'AvailabilityZone': 'us-west-2a', 'PreferredMaintenanceWindow': 'sat:06:30-sat:07:00', 'PendingModifiedValues': {}, 'ClusterVersion': '1.0', 'AllowVersionUpgrade': True, 'NumberOfNode